# Data Science Capstone, Part 3:
### Clustering Toronto's Boroughs based on type of venues

In this final notebook for the assignment of week 3 in the Data Science Capstone project course by IBM on Coursera I will use foursquare.com and its api to obtain top 10 type of venues (according to their frequency) in the different neighbourhoods of central Toronto. After having obtained the data related to venues per postcode, k-means will be used to cluster different postcodes.

This notebook is made after the one by Alex Alkson and Polong Lin hosted on IBM's skill networks server.

In [4]:
# Import required libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


try:
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
except ModuleNotFoundError:
    print("Library geopy not found: installing it before continuing")
    !pip install geopy
    from geopy.geocoders import Nominatim
    
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

try:
    import folium # map rendering library
except ModuleNotFoundError:
    print("Library folium not found: installing it before continuing")
    !pip install folium
    import folium

print('Libraries imported.')

Library geopy not found: installing it before continuing
Library folium not found: installing it before continuing
Libraries imported.


In [5]:
# Load dataframe obtained in the previous notebook, containing postcodes, borough, neighbourhoods and geo-coordinates

toronto_postcodes = pd.read_csv("TorontoPostGeo.csv", index_col=0)

toronto_postcodes.head() # Check the dataframe loaded correctly

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>to_explorer</em>, as shown below.

In [7]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with postcodes superimposed on top.

In [16]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for row in toronto_postcodes.iterrows():
    label = '{}, {}'.format(row[1]['Postal Code'], row[1]['Borough'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row[1]['Latitude'],row[1]['Longitude']],
        radius=5,
        popup = label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=.75,
        parse_html=False
    ).add_to(toronto_map)
    
toronto_map

#### Filter the original dataframe to limit the analysis to postcodes of Borough including the word "Toronto"

For the purpose of this analysis, we will limit the search to postcodes in central Toronto areas, therefore filtering the original dataset for boroughs containing the word Toronto in them.

In [27]:
# Filter the dataframe by converting the Borough column to lower case and then check which entry contains the word 'toronto'

central_toronto_postcodes = toronto_postcodes[toronto_postcodes['Borough'].str.lower().str.contains("toronto")]
central_toronto_postcodes.reset_index(drop=True, inplace=True)
central_toronto_postcodes.head() # Check the result

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547


#### Create new dataframe including all neighbourhoods on separate rows
In order to analyse the individual neighbourhoods, a new dataframe needs to be created, where each row contain just one of them (at the present moment, a row might contain multiple neighbourhoods if they share the same postcode prefix).
To do so, we iterate through the central_toronto_postcodes dataframe and, if multiple neighbourhoods are included, we split them with the split method and append each individual neighbourhood with the relative postcode and borough to the corresponding field in a dictionary. Once finished, that dictionary can be simply passed to pandas dataframe function to generate the new dataframe.

In [28]:
# Create an empty dictionary with keys corresponding to what will be the columns in the new dataframe
toronto_neighbourhoods = {'Postal Code':[],'Borough':[],'Neighbourhood':[]}

# Iterate through the central_toronto_postcodes dataframe to split the neighbourhoods and populate the dictionary
for row in central_toronto_postcodes.iterrows():
    for neighbourhood in row[1]['Neighbourhood'].split(', '):
        toronto_neighbourhoods['Postal Code'].append(row[1]['Postal Code'])
        toronto_neighbourhoods['Borough'].append(row[1]['Borough'])
        toronto_neighbourhoods['Neighbourhood'].append(neighbourhood)

# Convert the dictionary to a pandas dataframe
toronto_neigh =  pd.DataFrame(toronto_neighbourhoods)

# Check the difference in length between the central_toronto_postcodes dataframe and the newly created one
print("Postcode dataframe length = {}. Neighbourhood dataframe length = {}".format(len(central_toronto_postcodes),
                                                                                  len(toronto_neigh)))

toronto_neigh.head() # check the new dataframe

Postcode dataframe length = 39. Neighbourhood dataframe length = 78


,Postal Code,Borough,Neighbourhood
0,M5A,Downtown Toronto,Regent Park
1,M5A,Downtown Toronto,Harbourfront
2,M7A,Downtown Toronto,Queen's Park
3,M7A,Downtown Toronto,Ontario Provincial Government
4,M5B,Downtown Toronto,Garden District


#### Obtain the latitude and longitude for the neighbourhoods
At this point we need the exact coordinates for the individual neighbourhoods: these can be obtained by re-using the code from the previous notebook, but using geopy instead (as the library is already imported, otherwise geocoder can be imported and used as well).

In [38]:
# Create empty lists to be filled with geo-coordinates
latitudes = []
longitudes = []

# Loop through the neighbourhood dataframe, find coordinates for each neighbourhood and add them to the above lists
for neighbourhood in toronto_neigh['Neighbourhood']:
    
    address = '{}, Toronto, Ontario'.format(neighbourhood)
    print("Finding coordinates for {}...".format(address), end="")
    
    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    
    # If returned value = None, assign NaN to neighbourhood (later will be deleted)
    if location is None:
        latitude = np.nan
        longitude = np.nan
    else:
        latitude = location.latitude
        longitude = location.longitude
    latitudes.append(latitude)
    longitudes.append(longitude)
    print('Got them!')
    
# Add latitude and longitude to the neighbourhood dataframe
toronto_neigh['Latitude'] = latitudes
toronto_neigh['Longitude'] = longitudes

toronto_neigh.head() # check the result

Finding coordinates for Regent Park, Toronto, Ontario...Got them!
Finding coordinates for Harbourfront, Toronto, Ontario...Got them!
Finding coordinates for Queen's Park, Toronto, Ontario...Got them!
Finding coordinates for Ontario Provincial Government, Toronto, Ontario...Got them!
Finding coordinates for Garden District, Toronto, Ontario...Got them!
Finding coordinates for Ryerson, Toronto, Ontario...Got them!
Finding coordinates for St. James Town, Toronto, Ontario...Got them!
Finding coordinates for The Beaches, Toronto, Ontario...Got them!
Finding coordinates for Berczy Park, Toronto, Ontario...Got them!
Finding coordinates for Central Bay Street, Toronto, Ontario...Got them!
Finding coordinates for Christie, Toronto, Ontario...Got them!
Finding coordinates for Richmond, Toronto, Ontario...Got them!
Finding coordinates for Adelaide, Toronto, Ontario...Got them!
Finding coordinates for King, Toronto, Ontario...Got them!
Finding coordinates for Dufferin, Toronto, Ontario...Got them!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
1,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
2,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340
3,M7A,Downtown Toronto,Ontario Provincial Government,NaN,NaN
4,M5B,Downtown Toronto,Garden District,43.656500,-79.377114


#### Check how many neighbourhoods don't have geo-coordinates

In [44]:
toronto_neigh.isnull()['Latitude'].value_counts()

False    73
True      5
Name: Latitude, dtype: int64

As just five neighbourhoods miss location data, we can just drop them and proceed with analysing the other ones

In [46]:
toronto_neigh.dropna(axis=0, inplace=True) # drop rows containing NaN
toronto_neigh.reset_index(drop=True, inplace=True) # reset the index counter
toronto_neigh.head() # check results

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
1,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
2,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340
3,M5B,Downtown Toronto,Garden District,43.656500,-79.377114
4,M5B,Downtown Toronto,Ryerson,43.658469,-79.378993


#### Create new map showing the neighbourhoods of central Toronto
Replicate the above code for visualising Toronto, but this time highlighting the neighbourhoods of central Toronto

In [48]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

central_toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for row in toronto_neigh.iterrows():
    label = '{}, {}'.format(row[1]['Neighbourhood'], row[1]['Borough'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row[1]['Latitude'],row[1]['Longitude']],
        radius=5,
        popup = label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=.75,
        parse_html=False
    ).add_to(central_toronto_map)
    
central_toronto_map

By looking at the map and doing some research, we can see that the neighbourhood of Richmond and Underground City seem to have mistakenly entered in the map. We further clean the dataset by removing them.

In [49]:
# Get the indeces of Richmond and Underground city rows

indexNames = toronto_neigh[(toronto_neigh['Neighbourhood'] == 'Richmond')
                | (toronto_neigh['Neighbourhood'] == 'Underground city')].index

In [53]:
# Drop the above indeces from the dataframe

print('Lenght before dropping: {}'.format(len(toronto_neigh))) # print the length of the dataframe before and after to make sure that just 2 rows were dropped
toronto_neigh.drop(indexNames, axis=0, inplace=True) # drop Richmond and Underground city
toronto_neigh.reset_index(drop=True, inplace=True) # reset the index counter
print('Lenght after dropping: {}'.format(len(toronto_neigh)))


Lenght before dropping: 73
Lenght after dropping: 71


#### Connect to foursquare api
After having cleaned the dataframe and individuated the main objectives, we can finally proceed to obtain venues data for the analysed neighbourhoods. First, we connect to foursquare with the personal credentials

In [57]:
CLIENT_ID = 'ZIOD2EFJF5VTEDE3GK1VYGJ0P0WRJE3UVZ1RXOBFMFGB5VT0' # your Foursquare ID
CLIENT_SECRET = 'N3RQ2KSOLN3XOCYHZVB122DSPS4BR3RO4CI24VJ2ZBMSIUVT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Define the function to connect to foursquare api and get nearby values for each neighbourhood (function taken from the original notebook mentioned at the beginning)

In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
# Apply the above functions to each neighbourhood in central Toronto

toronto_venues = getNearbyVenues(names=toronto_neigh['Neighbourhood'],
                                   latitudes=toronto_neigh['Latitude'],
                                   longitudes=toronto_neigh['Longitude']
                                  )

toronto_venues.head() # check results

Regent Park
Harbourfront
Queen's Park
Garden District
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Dufferin
Dovercourt Village
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
The Danforth West
Riverdale
Toronto Dominion Centre
Design Exchange
Brockton
Parkdale Village
Exhibition Place
India Bazaar
The Beaches West
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
Forest Hill Road Park
High Park
The Junction South
North Toronto West
Lawrence Park
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
University of Toronto
Harbord
Runnymede
Swansea
Moore Park
Summerhill East
Kensington Market
Chinatown
Grange Park
Summerhill West
Rathnelly
South Hill
Forest Hill SE
Deer Park
CN Tower
King and Spadina
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Rosedale
St. James Town
Cabbagetown
First Canadian Place
Church and Wellesley


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,Regent Park,43.660706,-79.360457,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
2,Regent Park,43.660706,-79.360457,Sumach Espresso,43.658135,-79.359515,Coffee Shop
3,Regent Park,43.660706,-79.360457,Thai To Go,43.663418,-79.360710,Thai Restaurant
4,Regent Park,43.660706,-79.360457,Paintbox Bistro,43.660050,-79.362855,Restaurant


#### Check the resulting dataframe

In [60]:
toronto_venues.shape # check overall shape (number of rows, number of columns)

(3710, 7)

In [61]:
toronto_venues.groupby('Neighborhood').count() # check number of venues per neighbourhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,26,26,26,26,26,26
Berczy Park,100,100,100,100,100,100
Brockton,19,19,19,19,19,19
CN Tower,61,61,61,61,61,61
Cabbagetown,52,52,52,52,52,52
Central Bay Street,41,41,41,41,41,41
Chinatown,70,70,70,70,70,70
Christie,58,58,58,58,58,58


#### Encode the venues by category with one hot encoding

In [62]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,T

In [64]:
# Let's see how many categories are there (number of columns of toronto_onehot - 1)

print("Unique venues' categories in central Toronto: {}".format(toronto_onehot.shape[1]-1))

Unique venues' categories in central Toronto: 302


#### Aggregating venues' categories by neighbourhood (using mean as aggregator function)

In [65]:
toronto_aggregated_venues = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_aggregated_venues.head(15)

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,T

#### Printing each neighborhood along with the top 5 most common venues

In [66]:
num_top_venues = 5

for hood in toronto_aggregated_venues['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_aggregated_venues[toronto_aggregated_venues['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.07
1           Restaurant  0.05
2            Gastropub  0.04
3  Japanese Restaurant  0.04
4                 Café  0.04


----Bathurst Quay----
              venue  freq
0       Coffee Shop  0.15
1              Park  0.12
2              Café  0.12
3   Harbor / Marina  0.08
4  Ramen Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.06
2               Hotel  0.05
3                Café  0.05
4  Italian Restaurant  0.04


----Brockton----
                   venue  freq
0                    Bar  0.16
1                   Park  0.11
2  Vietnamese Restaurant  0.11
3  Portuguese Restaurant  0.05
4      Korean Restaurant  0.05


----CN Tower----
              venue  freq
0             Hotel  0.11
1       Coffee Shop  0.10
2       Pizza Place  0.05
3  Baseball Stadium  0.05
4    Scenic Lookout  0.03


----Cabbagetown----
         venue  freq
0   Restaurant  

4          Grocery Store  0.07


----Roncesvalles----
            venue  freq
0            Café  0.08
1      Restaurant  0.05
2     Art Gallery  0.05
3  Breakfast Spot  0.05
4       Bookstore  0.05


----Rosedale----
        venue  freq
0        Park  0.33
1  Playground  0.33
2  Bike Trail  0.33
3        Pool  0.00
4   Nightclub  0.00


----Roselawn----
                    venue  freq
0             Coffee Shop  0.18
1                    Bank  0.09
2  Argentinian Restaurant  0.09
3                    Park  0.09
4               BBQ Joint  0.09


----Runnymede----
         venue  freq
0         Café  0.10
1  Coffee Shop  0.10
2       Bakery  0.08
3         Bank  0.05
4  Pizza Place  0.05


----Ryerson----
                 venue  freq
0          Coffee Shop  0.13
1       Clothing Store  0.05
2                 Café  0.04
3  Japanese Restaurant  0.03
4         Burger Joint  0.03


----South Hill----
               venue  freq
0     Sandwich Place  0.11
1     History Museum  0.11
2           

#### Defining function to return most common venues in central Toronto (from original notebook mentioned at the beginning)

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Applying above function to toronto_onehot

In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_aggregated_venues['Neighborhood']

for ind in np.arange(toronto_aggregated_venues.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_aggregated_venues.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Japanese Restaurant,Gastropub,Café,Cosmetics Shop,Gym,Clothing Store,American Restaurant,Seafood Restaurant
1,Bathurst Quay,Coffee Shop,Café,Park,Harbor / Marina,Sushi Restaurant,Japanese Restaurant,Gym,Ramen Restaurant,Tunnel,Diner
2,Berczy Park,Coffee Shop,Restaurant,Hotel,Café,Japanese Restaurant,Italian Restaurant,Gym,Gastropub,Seafood Restaurant,Bakery
3,Brockton,Bar,Vietnamese Restaurant,Park,Korean Restaurant,Dive Bar,Auto Workshop,Gastropub,Sake Bar,Jazz Club,Grocery Store
4,CN Tower,Hotel,Coffee Shop,Pizza Place,Baseball Stadium,Restaurant,Italian Restaurant,Ice Cream Shop,Gym,Concert Hall,Scenic Lookout


As it could maybe be expected, central Toronto seems to be a place in which hospitality venues are quite popular

### Clustering Central Toronto Neighbourhoods

In [71]:
# set number of clusters
kclusters = int(input("Enter number of clusters to be used for k-means: "))

toronto4clustering = toronto_aggregated_venues.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto4clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

Enter number of clusters to be used for k-means: 5


array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2])

### creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [76]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neigh

toronto_merged.columns = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] # make sure column names will match

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,2,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Animal Shelter,Sushi Restaurant,Auto Dealership,Electronics Store,Beer Store,Restaurant,Food Truck
1,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Gym,Bank,Music Venue,Sushi Restaurant
2,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340,2,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Japanese Restaurant,French Restaurant,Bubble Tea Shop,Restaurant,Thai Restaurant,Juice Bar
3,M5B,Downtown Toronto,Garden District,43.656500,-79.377114,2,Clothing Store,Coffee Shop,Restaurant,Hotel,Theater,Movie Theater,Café,Sandwich Place,Electronics Store,Bookstore
4,M5B,Downtown Toronto,Ryerson,43.658469,-79.378993,2,Coffee Shop,Clothing Store,Café,Burger Joint,Japanese Restaurant,Movie Theater,Bubble Tea Shop,Ramen Restaurant,Pizza Place,Middle Eastern Restaurant


Cluster number 2 definetely seem to include neighborhoods where coffee shops are quite popular

#### Creating map for visualising clusters

In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Qualitatively Assessing what the different clusters contain in terms of venues 

Now that the clustering has been done, we can look at what type of venues is contained in each cluster so as to have an idea of what might be in common among neighborhoods sharing the same clusters.

In [114]:
# defining column names where top venues features are contained

filtering_columns = [column for column in toronto_merged.columns.values if column.endswith('Venue')]

# defining dictionaries containing the results
clusters = {}
length_neighbours = {} # this dictionary will contain how many neighborhoods each cluster contains

# Loop through the clusters, filter by cluster number and obtain the top 10 venue categories per cluster
for cluster in range(kclusters):
    venue_dictionary = {}
    
    top_venues_dataframe = toronto_merged[toronto_merged['Cluster Labels']==cluster][filtering_columns]
    length_neighbours[cluster] = len(top_venues_dataframe)
    
    for column in top_venues_dataframe:
        for row in pd.DataFrame(top_venues_dataframe[column].value_counts()).reset_index(level=0).iterrows():
            if row[1]['index'] not in venue_dictionary:
                venue_dictionary[row[1]['index']] = row[1][column]
            else:
                venue_dictionary[row[1]['index']] += row[1][column]
    
    # Sort the dictionary by value (i.e. count of specific venue category)
    i = 0
    top_sorted_dict = {}
    sorted_dict = {key:value for key,value in sorted(venue_dictionary.items(), key=lambda x: x[1],reverse=True)}
    print('Top 10 venues for cluster number {} (normalised by number of neighbours)'.format(cluster))
    for key,value in sorted_dict.items():
        i+=1
        if i>10:
            break
        else:
            print("  ", key, value)
            top_sorted_dict[key] = value
    
    print('\n')
    clusters[cluster] = sorted_dict

Top 10 venues for cluster number 0 (normalised by number of neighbours)
   Café 3
   Park 3
   Coffee Shop 2
   Harbor / Marina 2
   Music Venue 1
   Convenience Store 1
   Women's Store 1
   Sushi Restaurant 1
   Bar 1
   Escape Room 1


Top 10 venues for cluster number 1 (normalised by number of neighbours)
   Playground 2
   Park 2
   Ethiopian Restaurant 2
   Dumpling Restaurant 2
   Eastern European Restaurant 2
   Egyptian Restaurant 2
   Electronics Store 2
   Escape Room 2
   Bike Trail 1
   Bank 1


Top 10 venues for cluster number 2 (normalised by number of neighbours)
   Coffee Shop 52
   Café 41
   Restaurant 30
   Bar 25
   Italian Restaurant 24
   Japanese Restaurant 21
   Sushi Restaurant 19
   Hotel 17
   Pizza Place 15
   Bakery 13


Top 10 venues for cluster number 3 (normalised by number of neighbours)
   Park 1
   Pilates Studio 1
   Dance Studio 1
   Women's Store 1
   Event Space 1
   Dumpling Restaurant 1
   Eastern European Restaurant 1
   Egyptian Restaurant 1


By looking at the results above, we can conclude that the first cluster (Cluster number 0) is notable mainly for parks and include probably neighborhoods by the sea (considering the harbor/marina category).
***
Cluster number 1 distinguish itself by the number of ethnic restaurants as well as playgrounds and parks.
***
Cluster number 2 seems to be the one with the most neighborhoods and, in general, has many coffee shops and a notable number of hotels, probably representing parts of Toronto in which there is high mobility and potentially tourism
***
Cluster 3 seems to have just one example of different categories, so that no particular pattern is truly identifiable
***
Cluster 4, finally, might be a more residential area having 2 electronics store and trails and less coffee shops than, for example, cluster 2

In [116]:
# Now let's look at how many neighborhoods each cluster contains
for key, value in length_neighbours.items():
    print('Cluster number {} contains {} neighborhoods'.format(key, value))

Cluster number 0 contains 3 neighborhoods
Cluster number 1 contains 2 neighborhoods
Cluster number 2 contains 63 neighborhoods
Cluster number 3 contains 1 neighborhoods
Cluster number 4 contains 2 neighborhoods


By looking at the actual number of neighborhoods per cluster, finally, it seems that cluster number 2 contains most of the neighbourhoods. The other four clusters, then, seem to represent more particular oursiders than general patterns, given the small number of neighborhoods in them.